# 라이브러리 로드

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

import warnings
from pathlib import Path

from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

import xgboost as xgb

from hyperopt import hp 
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials, space_eval

In [3]:
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)

warnings.filterwarnings('ignore')

# 학습데이터 로드

In [4]:
data_dir = Path('../data/dacon-novel-author-classification')
feature_dir = Path('../build/feature')
val_dir = Path('../build/val')
tst_dir = Path('../build/tst')
sub_dir = Path('../build/sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5
seed = 2020

In [5]:
algorithm_name = 'xgb'

feature_names= ['stacking-layer2']

feature_target_file = feature_dir / f'feature_target.csv'

model_names = []
for feature_name in feature_names:
    model_names.append(f'{algorithm_name}_{feature_name}')
    
stacking_oof_pred_files=[]
for model_name in model_names:
    stacking_oof_pred_files.append( val_dir / f'{model_name}_oof_pred.csv')
    
stacking_test_pred_files=[]
for model_name in model_names:
    stacking_test_pred_files.append( tst_dir / f'{model_name}_test_pred.csv')
    
stacking_submission_files=[]
for model_name in model_names:
    stacking_submission_files.append( sub_dir / f'{model_name}_submission.csv')

# Stacking feature 생성

In [6]:
def load_feature(model_names, number_of_ver=None, kind=None):
    oof_list = []
    test_list = []
    
    if number_of_ver==None or kind==None:
        print('error')
        return None
    
    # 딥러닝 시리즈 4가지 버전
    if kind == 0:
        for model in model_names:
            print(f'load {model}_cv')
            for i in range(1,number_of_ver+1):
                oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv', delimiter=','))
                test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
    
    # 로지스틱 회귀 6가지 버전
    elif kind == 1:
        for model in model_names:
            print(f'load {model}_cv')
            for i in range(1, number_of_ver+1):
                oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv', delimiter=','))
                test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))

    # 신경망 기반 불용어 처리 21가지 버전 또는 머신러닝 기반 불용어 처리 18가지 버전
    elif kind == 2:
        for model in model_names:
            print(f'load {model}_cv')
            if model.find('feature') != -1:
                for i in range(2,5):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('tfidf') != -1:
                for i in range(1,4):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('hashing') != -1:
                for i in range(1,4):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('bow') != -1:
                for i in range(1,4):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            else:
                print('not found')
    
    # 신경만 기반 불용어 처리 X 13가지 버전 또는 머신러닝 기반 불용어 처리 X 18가지 버전
    elif kind == 3:
        for model in model_names:
            print(f'load {model}_cv')
            if model.find('feature') != -1:
                for i in range(1,2):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('tfidf') != -1:
                for i in range(4,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('hashing') != -1:
                for i in range(4,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('bow') != -1:
                for i in range(4,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            else:
                print('not found')
                
    # 첫번째 레이어를 학습하기 위한 데이터셋 모두 가져오기 버전
    elif kind == 4:
        for model in model_names:
            print(f'load {model}_cv')
            if model.find('feature') != -1:
                for i in range(1,5):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('tfidf') != -1:
                for i in range(1,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('hashing') != -1:
                for i in range(1,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            elif model.find('bow') != -1:
                for i in range(1,7):
                    oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver{i}.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred_ver{i}.csv', delimiter=','))
            else:
                print('not found')
    
    # 두번째 레이어를 학습하기 위한 데이터셋 모두 가져오기 버전
    elif kind == 5:
        for model in model_names:
            print(f'load {model}_cv')
            if model.find('stacking') != -1:
                for feature in ['stopwords-yes-nn','stopwords-no-nn','stopwords-no-ml', 'stopwords-no-ml'] :
                    oof_list.append(np.loadtxt(val_dir / f'{model}-{feature}_oof_pred.csv',delimiter=','))
                    test_list.append(np.loadtxt(tst_dir / f'{model}-{feature}_test_pred.csv',delimiter=','))
            elif model.find('all') != -1:
                oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred.csv',delimiter=','))
                test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred.csv',delimiter=','))
    
    # 두번째 레이어를 학습하기 위한 PCA 데이터셋 + xgb/lgbm tfidf 데이터셋 가져오기 버전 - 하나씩
    elif kind ==6:
        for model in model_names:
            print(f'load {model}_cv')
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred.csv', delimiter=','))
            test_list.append(np.loadtxt(tst_dir / f'{model}_test_pred.csv', delimiter=','))
            
                        
        
    
    return oof_list, test_list

In [7]:
model_names = ['mlp_tfidf-pca', 'lr_tfidf-pca', 'cnn_tfidf-pca', 'lgbm_tfidf','lgbm_tfidf-pca']

tmp_oof, tmp_test = load_feature(model_names, -1, 6)
tmp_oof = np.concatenate(tmp_oof, axis=1)
tmp_test = np.concatenate(tmp_test, axis=1)
print(f'shape : {tmp_oof.shape}, {tmp_test.shape}')

load mlp_tfidf-pca_cv
load lr_tfidf-pca_cv
load cnn_tfidf-pca_cv
load lgbm_tfidf_cv
load lgbm_tfidf-pca_cv
shape : (54879, 25), (19617, 25)


In [8]:
model_names = ['ets_stacking-layer1','gb_stacking-layer1','lgbm_stacking-layer1','mlp_stacking-layer1',
              'rf_stacking-layer1','xgb_stacking-layer1','ada_stacking-layer1',
              'xgb_all','lgbm_all']

all_oof, all_test = load_feature(model_names, -1, 5)
all_oof = np.concatenate(all_oof, axis=1)
all_test = np.concatenate(all_test, axis=1)
print(f'shape : {all_oof.shape}, {all_test.shape}')

load ets_stacking-layer1_cv
load gb_stacking-layer1_cv
load lgbm_stacking-layer1_cv
load mlp_stacking-layer1_cv
load rf_stacking-layer1_cv
load xgb_stacking-layer1_cv
load ada_stacking-layer1_cv
load xgb_all_cv
load lgbm_all_cv
shape : (54879, 150), (19617, 150)


In [9]:
all_oof = np.concatenate([all_oof, tmp_oof], axis=1)
all_test = np.concatenate([all_test, tmp_test], axis=1)
print(f'shape : {all_oof.shape}, {all_test.shape}')

shape : (54879, 175), (19617, 175)


In [10]:
y = pd.read_csv(feature_target_file, index_col=0, usecols=['index',target_col]).values.flatten()
y.shape

(54879,)

# 하이퍼파라미터 튜닝

In [11]:
# XGB parameters

xgb_reg_params = {
    'n_jobs': -1,
    'n_estimators': 1000,
    'eval_metric': 'mlogloss',
    'objective': 'multi:softprob',
    'num_class': 5,
    'random_state': 2020,
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',

    'booster': 'gbtree',

    'max_depth': hp.choice('max_depth',np.arange(5,30,1,dtype=int)),
    'alpha': hp.uniform('alpha', 0.01, 0.4),
    'lambda': hp.uniform('lambda', 0.01, 0.4),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 0.9),
    'gamma': hp.uniform('gamma', 0.01, 0.7),
    'max_leaves': hp.choice('max_leaves', np.arange(20,250,1,dtype=int)),
    'subsample': hp.choice('subsample', [0.2,0.4,0.5,0.6,0.7,0.8,0.9]),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.4, 0.8),
}

xgb_fit_params = {
    'eval_metric': 'mlogloss',
    'early_stopping_rounds': 50,
    'verbose': True
}

xgb_para = dict()
xgb_para['reg_params'] = xgb_reg_params
xgb_para['fit_params'] = xgb_fit_params
xgb_para['loss_func'] = lambda y, pred: log_loss(y,pred, labels=[0,1,2,3,4]) 

In [12]:
class HPOpt(object):

    def __init__(self, X_train, X_val, y_train, y_val):
        self.X_train = X_train
        self.X_val = X_val
        self.y_train = y_train
        self.y_val = y_val

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result

    def xgb_clf(self, para):
        clf = xgb.XGBClassifier(**para['reg_params'])
        return self.train_reg(clf, para)
    
    def train_reg(self, clf, para):
        clf.fit(self.X_train, self.y_train,
                eval_set=[(self.X_train, self.y_train), (self.X_val, self.y_val)],
                **para['fit_params'])
        
        loss = clf.best_score
        return {'loss': loss, 'status': STATUS_OK, 'model': clf}

In [13]:
X_train, X_val, y_train, y_val = train_test_split(all_oof, y, test_size= 0.25, random_state=2020)

obj = HPOpt(X_train, X_val, y_train, y_val)

trials=Trials()
best = obj.process(fn_name='xgb_clf', space=xgb_para, trials=trials, algo=tpe.suggest, max_evals=30)

[0]	validation_0-mlogloss:1.32233	validation_1-mlogloss:1.33320

Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.


Will train until validation_1-mlogloss hasn't improved in 50 rounds.

[1]	validation_0-mlogloss:1.12593	validation_1-mlogloss:1.14592

[2]	validation_0-mlogloss:0.97870	validation_1-mlogloss:1.00691

[3]	validation_0-mlogloss:0.86156	validation_1-mlogloss:0.89803

[4]	validation_0-mlogloss:0.76837	validation_1-mlogloss:0.81238

[5]	validation_0-mlogloss:0.69249	validation_1-mlogloss:0.74305

[6]	validation_0-mlogloss:0.62895	validation_1-mlogloss:0.68616

[7]	validation_0-mlogloss:0.57611	validation_1-mlogloss:0.63948

[8]	validation_0-mlogloss:0.53072	validation_1-mlogloss:0.60030

[9]	validation_0-mlogloss:0.49193	validation_1-mlogloss:0.56780

[10]	validation_0-mlogloss:0.45897	validation_1-mlogloss:0.54044

[11]	validation_0-mlogloss:0.43077	validation_1-mlogloss:0.51778

[12]	validation_0-mlogloss:0.40594	validation_1-m

[25]	validation_0-mlogloss:0.20024	validation_1-mlogloss:0.40708      

[26]	validation_0-mlogloss:0.19341	validation_1-mlogloss:0.40608      

[27]	validation_0-mlogloss:0.18673	validation_1-mlogloss:0.40501      

[28]	validation_0-mlogloss:0.18032	validation_1-mlogloss:0.40432      

[29]	validation_0-mlogloss:0.17445	validation_1-mlogloss:0.40384      

[30]	validation_0-mlogloss:0.16848	validation_1-mlogloss:0.40368      

[31]	validation_0-mlogloss:0.16317	validation_1-mlogloss:0.40322      

[32]	validation_0-mlogloss:0.15819	validation_1-mlogloss:0.40326      

[33]	validation_0-mlogloss:0.15338	validation_1-mlogloss:0.40334      

[34]	validation_0-mlogloss:0.14838	validation_1-mlogloss:0.40370      

[35]	validation_0-mlogloss:0.14350	validation_1-mlogloss:0.40407      

[36]	validation_0-mlogloss:0.13892	validation_1-mlogloss:0.40423      

[37]	validation_0-mlogloss:0.13430	validation_1-mlogloss:0.40445      

[38]	validation_0-mlogloss:0.13006	validation_1-mlogloss:0.40484

[52]	validation_0-mlogloss:0.24992	validation_1-mlogloss:0.41041      

[53]	validation_0-mlogloss:0.24635	validation_1-mlogloss:0.40937      

[54]	validation_0-mlogloss:0.24298	validation_1-mlogloss:0.40835      

[55]	validation_0-mlogloss:0.23959	validation_1-mlogloss:0.40762      

[56]	validation_0-mlogloss:0.23639	validation_1-mlogloss:0.40691      

[57]	validation_0-mlogloss:0.23317	validation_1-mlogloss:0.40602      

[58]	validation_0-mlogloss:0.23010	validation_1-mlogloss:0.40535      

[59]	validation_0-mlogloss:0.22705	validation_1-mlogloss:0.40471      

[60]	validation_0-mlogloss:0.22417	validation_1-mlogloss:0.40417      

[61]	validation_0-mlogloss:0.22127	validation_1-mlogloss:0.40369      

[62]	validation_0-mlogloss:0.21851	validation_1-mlogloss:0.40321      

[63]	validation_0-mlogloss:0.21596	validation_1-mlogloss:0.40278      

[64]	validation_0-mlogloss:0.21329	validation_1-mlogloss:0.40236      

[65]	validation_0-mlogloss:0.21080	validation_1-mlogloss:0.40203

[26]	validation_0-mlogloss:0.82818	validation_1-mlogloss:0.83356      

[27]	validation_0-mlogloss:0.81330	validation_1-mlogloss:0.81890      

[28]	validation_0-mlogloss:0.79910	validation_1-mlogloss:0.80492      

[29]	validation_0-mlogloss:0.78537	validation_1-mlogloss:0.79148      

[30]	validation_0-mlogloss:0.77216	validation_1-mlogloss:0.77850      

[31]	validation_0-mlogloss:0.75921	validation_1-mlogloss:0.76591      

[32]	validation_0-mlogloss:0.74682	validation_1-mlogloss:0.75378      

[33]	validation_0-mlogloss:0.73484	validation_1-mlogloss:0.74206      

[34]	validation_0-mlogloss:0.72318	validation_1-mlogloss:0.73062      

[35]	validation_0-mlogloss:0.71199	validation_1-mlogloss:0.71959      

[36]	validation_0-mlogloss:0.70117	validation_1-mlogloss:0.70905      

[37]	validation_0-mlogloss:0.69077	validation_1-mlogloss:0.69890      

[38]	validation_0-mlogloss:0.68070	validation_1-mlogloss:0.68912      

[39]	validation_0-mlogloss:0.67083	validation_1-mlogloss:0.67955

[139]	validation_0-mlogloss:0.36283	validation_1-mlogloss:0.40348     

[140]	validation_0-mlogloss:0.36207	validation_1-mlogloss:0.40304     

[141]	validation_0-mlogloss:0.36132	validation_1-mlogloss:0.40260     

[142]	validation_0-mlogloss:0.36055	validation_1-mlogloss:0.40219     

[143]	validation_0-mlogloss:0.35987	validation_1-mlogloss:0.40182     

[144]	validation_0-mlogloss:0.35915	validation_1-mlogloss:0.40144     

[145]	validation_0-mlogloss:0.35842	validation_1-mlogloss:0.40114     

[146]	validation_0-mlogloss:0.35771	validation_1-mlogloss:0.40087     

[147]	validation_0-mlogloss:0.35701	validation_1-mlogloss:0.40056     

[148]	validation_0-mlogloss:0.35632	validation_1-mlogloss:0.40029     

[149]	validation_0-mlogloss:0.35570	validation_1-mlogloss:0.39997     

[150]	validation_0-mlogloss:0.35508	validation_1-mlogloss:0.39969     

[151]	validation_0-mlogloss:0.35438	validation_1-mlogloss:0.39935     

[152]	validation_0-mlogloss:0.35375	validation_1-mlogloss:0.3990

...

[146]	validation_0-mlogloss:0.44378	validation_1-mlogloss:0.52285      

[147]	validation_0-mlogloss:0.44148	validation_1-mlogloss:0.52099      

[148]	validation_0-mlogloss:0.43922	validation_1-mlogloss:0.51919      

[149]	validation_0-mlogloss:0.43697	validation_1-mlogloss:0.51745      

[150]	validation_0-mlogloss:0.43477	validation_1-mlogloss:0.51572      

[151]	validation_0-mlogloss:0.43257	validation_1-mlogloss:0.51401      

[152]	validation_0-mlogloss:0.43042	validation_1-mlogloss:0.51233      

[153]	validation_0-mlogloss:0.42832	validation_1-mlogloss:0.51064      

[154]	validation_0-mlogloss:0.42619	validation_1-mlogloss:0.50899      

[155]	validation_0-mlogloss:0.42407	validation_1-mlogloss:0.50740      

[156]	validation_0-mlogloss:0.42201	validation_1-mlogloss:0.50574      

[157]	validation_0-mlogloss:0.41997	validation_1-mlogloss:0.50418      

[158]	validation_0-mlogloss:0.41795	validation_1-mlogloss:0.50264      

[159]	validation_0-mlogloss:0.41596	validation_1-ml

[258]	validation_0-mlogloss:0.28987	validation_1-mlogloss:0.41928      

[259]	validation_0-mlogloss:0.28902	validation_1-mlogloss:0.41888      

[260]	validation_0-mlogloss:0.28818	validation_1-mlogloss:0.41845      

[261]	validation_0-mlogloss:0.28736	validation_1-mlogloss:0.41811      

[262]	validation_0-mlogloss:0.28656	validation_1-mlogloss:0.41776      

[263]	validation_0-mlogloss:0.28576	validation_1-mlogloss:0.41738      

[264]	validation_0-mlogloss:0.28498	validation_1-mlogloss:0.41700      

[265]	validation_0-mlogloss:0.28421	validation_1-mlogloss:0.41665      

[266]	validation_0-mlogloss:0.28340	validation_1-mlogloss:0.41631      

[267]	validation_0-mlogloss:0.28266	validation_1-mlogloss:0.41599      

[268]	validation_0-mlogloss:0.28192	validation_1-mlogloss:0.41563      

[269]	validation_0-mlogloss:0.28113	validation_1-mlogloss:0.41530      

[270]	validation_0-mlogloss:0.28035	validation_1-mlogloss:0.41499      

[271]	validation_0-mlogloss:0.27960	validation_1-ml

[370]	validation_0-mlogloss:0.22287	validation_1-mlogloss:0.39733      

[371]	validation_0-mlogloss:0.22243	validation_1-mlogloss:0.39729      

[372]	validation_0-mlogloss:0.22197	validation_1-mlogloss:0.39720      

[373]	validation_0-mlogloss:0.22154	validation_1-mlogloss:0.39715      

[374]	validation_0-mlogloss:0.22110	validation_1-mlogloss:0.39707      

[375]	validation_0-mlogloss:0.22066	validation_1-mlogloss:0.39703      

[376]	validation_0-mlogloss:0.22022	validation_1-mlogloss:0.39696      

[377]	validation_0-mlogloss:0.21980	validation_1-mlogloss:0.39688      

[378]	validation_0-mlogloss:0.21937	validation_1-mlogloss:0.39682      

[379]	validation_0-mlogloss:0.21891	validation_1-mlogloss:0.39678      

[380]	validation_0-mlogloss:0.21848	validation_1-mlogloss:0.39674      

[381]	validation_0-mlogloss:0.21805	validation_1-mlogloss:0.39670      

[382]	validation_0-mlogloss:0.21764	validation_1-mlogloss:0.39666      

[383]	validation_0-mlogloss:0.21720	validation_1-ml

[482]	validation_0-mlogloss:0.18096	validation_1-mlogloss:0.39454      

[483]	validation_0-mlogloss:0.18065	validation_1-mlogloss:0.39456      

[484]	validation_0-mlogloss:0.18033	validation_1-mlogloss:0.39456      

[485]	validation_0-mlogloss:0.17999	validation_1-mlogloss:0.39455      

[486]	validation_0-mlogloss:0.17968	validation_1-mlogloss:0.39455      

[487]	validation_0-mlogloss:0.17936	validation_1-mlogloss:0.39451      

[488]	validation_0-mlogloss:0.17905	validation_1-mlogloss:0.39454      

[489]	validation_0-mlogloss:0.17874	validation_1-mlogloss:0.39454      

[490]	validation_0-mlogloss:0.17842	validation_1-mlogloss:0.39451      

[491]	validation_0-mlogloss:0.17811	validation_1-mlogloss:0.39454      

[492]	validation_0-mlogloss:0.17780	validation_1-mlogloss:0.39454      

[493]	validation_0-mlogloss:0.17749	validation_1-mlogloss:0.39454      

[494]	validation_0-mlogloss:0.17716	validation_1-mlogloss:0.39454      

[495]	validation_0-mlogloss:0.17685	validation_1-ml

[77]	validation_0-mlogloss:0.34063	validation_1-mlogloss:0.38935       

[78]	validation_0-mlogloss:0.33988	validation_1-mlogloss:0.38928       

[79]	validation_0-mlogloss:0.33916	validation_1-mlogloss:0.38915       

[80]	validation_0-mlogloss:0.33831	validation_1-mlogloss:0.38912       

[81]	validation_0-mlogloss:0.33751	validation_1-mlogloss:0.38905       

[82]	validation_0-mlogloss:0.33687	validation_1-mlogloss:0.38909       

[83]	validation_0-mlogloss:0.33620	validation_1-mlogloss:0.38904       

[84]	validation_0-mlogloss:0.33549	validation_1-mlogloss:0.38907       

[85]	validation_0-mlogloss:0.33471	validation_1-mlogloss:0.38900       

[86]	validation_0-mlogloss:0.33406	validation_1-mlogloss:0.38898       

[87]	validation_0-mlogloss:0.33335	validation_1-mlogloss:0.38894       

[88]	validation_0-mlogloss:0.33264	validation_1-mlogloss:0.38891       

[89]	validation_0-mlogloss:0.33184	validation_1-mlogloss:0.38886       

[90]	validation_0-mlogloss:0.33101	validation_1-mlo

In [14]:
best

{'alpha': 0.39420323409725777,
 'colsample_bytree': 0.5584650715157261,
 'gamma': 0.23773305605324335,
 'lambda': 0.014292270434474535,
 'learning_rate': 0.010370218699913357,
 'max_depth': 17,
 'max_leaves': 13,
 'subsample': 1}

In [15]:
hyperparams = space_eval(xgb_reg_params, best)
hyperparams

{'alpha': 0.39420323409725777,
 'booster': 'gbtree',
 'colsample_bytree': 0.5584650715157261,
 'eval_metric': 'mlogloss',
 'gamma': 0.23773305605324335,
 'lambda': 0.014292270434474535,
 'learning_rate': 0.010370218699913357,
 'max_depth': 22,
 'max_leaves': 33,
 'n_estimators': 1000,
 'n_jobs': -1,
 'num_class': 5,
 'objective': 'multi:softprob',
 'predictor': 'gpu_predictor',
 'random_state': 2020,
 'subsample': 0.4,
 'tree_method': 'gpu_hist'}

In [16]:
trials.best_trial['result']['model']

XGBClassifier(alpha=0.39420323409725777, base_score=0.5, booster='gbtree',
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.5584650715157261, eval_metric='mlogloss',
              gamma=0.23773305605324335, gpu_id=0, importance_type='gain',
              interaction_constraints='', lambda=0.014292270434474535,
              learning_rate=0.010370218699913357, max_delta_step=0,
              max_depth=22, ma...
              monotone_constraints='(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)',
              n_estimators=1000, n_jobs=-1, num_class=5, num_parallel_tree=1,
              objective='multi:softprob', predictor='gpu_predictor',
          